In [1]:
from os import environ
# environ["AP_PORT"] = "25432" # str(input('port'))
# README command uses port=25432

In [2]:
import pickle
import json
import os
import psycopg2
import numpy as np

from spatialyze.database import database
from spatialyze.geospatial_video import GeospatialVideo
from spatialyze.road_network import RoadNetwork
from spatialyze.video_processor.camera_config import camera_config
from spatialyze.video_processor.stages.tracking_3d.tracking_3d import Tracking3DResult
from spatialyze.world import World, _execute
from spatialyze.video_processor.cache import disable_cache
from spatialyze.video_processor.metadata_json_encoder import MetadataJSONEncoder

CUDA is available.
 > 0: Tesla T4


In [3]:
OUTPUT_DIR = '/home/youse/spatialyze/data/pipeline/test-results'
VIDEO_DIR =  '/home/youse/viva-data' # '../../data/pipeline/videos'
# ROAD_DIR = '../../data/scenic/road-network/boston-seaport'

files = os.listdir(VIDEO_DIR)

disable_cache()

In [4]:
# database = Database(
#     psycopg2.connect(
#         dbname=environ.get("AP_DB", "mobilitydb"),
#         user=environ.get("AP_USER", "docker"),
#         host=environ.get("AP_HOST", "localhost"),
#         port=environ.get("AP_PORT", "25432"),
#         password=environ.get("AP_PASSWORD", "docker"),
#     )
# )

In [5]:
world = World(database)
# world.addGeogConstructs(RoadNetwork('Boston-Seaport', ROAD_DIR))

In [6]:
from pyquaternion import Quaternion

CAMERA_INTRINSIC = np.array([
    [1272,    0, 960],
    [   0, 1272, 540],
    [   0,    0,   1]
])

CAMERA_TRANSLATION = np.array([0, 0, 5])

CAMERA_ROTATION = Quaternion((0.430, -0.561, 0.561, -0.430))

In [7]:
import cv2
import datetime

for video in files:
    videofile = os.path.join(VIDEO_DIR, video)
    
    cap = cv2.VideoCapture(videofile)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    camera = []
    for frame in range(frame_count):
        camera.append(camera_config(
            camera_id=video,
            camera_heading=0,
            camera_intrinsic=CAMERA_INTRINSIC,
            camera_translation=CAMERA_TRANSLATION,
            ego_heading=0,
            ego_rotation=Quaternion((1, 0, 0, 0)),
            camera_rotation=CAMERA_ROTATION,
            filename=videofile,
            ego_translation=np.array([0, 0, 0]),
            frame_id=frame,
            frame_num=frame,
            location="viva-data",
            timestamp=datetime.datetime.fromtimestamp(frame + 10),
            road_direction=0,
        ))

    world.addVideo(GeospatialVideo(videofile, camera))

In [8]:
from spatialyze.utils import F

o = world.object()
# p = world.object()
c = world.camera()
world.filter(
    (o.type == 'car') &
    F.contained(o.trans@c.time, 'intersection') &
    F.left_turn(o)
)

In [9]:
import time

start = time.time()
result = world.getObjects()
end = time.time()


UndefinedColumn: column "__roadtype__intersection__" does not exist
LINE 7:                 AND "__roadtype__intersection__"
                            ^


In [ ]:
print("result", format(end-start))


result 1163.3296129703522


In [ ]:
result = world.getObjects()
result

[MovableObject(id=3, type='car', track=[(16.59044965664386, -5.323930717183163, 0.0), (16.59044965664386, -5.323930717183163, 0.0), (16.59044965664386, -5.323930717183163, 0.0), (16.59044965664386, -5.323930717183163, 0.0), (16.59044965664386, -5.317122621636894, 0.0), (16.59044965664386, -5.310314526090624, 0.0), (16.59044965664386, -5.3035064305443544, 0.0), (16.59044965664386, -5.3035064305443544, 0.0), (16.59044965664386, -5.2966983349980845, 0.0), (16.59044965664386, -5.2966983349980845, 0.0), (16.59044965664386, -5.3035064305443544, 0.0), (16.59044965664386, -5.2966983349980845, 0.0), (16.59044965664386, -5.310314526090624, 0.0), (16.59044965664386, -5.3035064305443544, 0.0), (16.59044965664386, -5.3035064305443544, 0.0), (16.59044965664386, -5.3035064305443544, 0.0), (16.59044965664386, -5.2966983349980845, 0.0), (16.59044965664386, -5.2966983349980845, 0.0), (16.637739943799524, -5.31748983547261, 0.0), (16.59044965664386, -5.296698334998083, 0.0), (16.59044965664386, -5.296698

In [13]:
world.saveVideos(outputDir=OUTPUT_DIR, addBoundingBoxes=True)

[('/home/youse/viva-data/5.mp4', 6),
 ('/home/youse/viva-data/5.mp4', 8),
 ('/home/youse/viva-data/5.mp4', 14),
 ('/home/youse/viva-data/5.mp4', 15),
 ('/home/youse/viva-data/5.mp4', 16),
 ('/home/youse/viva-data/5.mp4', 20),
 ('/home/youse/viva-data/5.mp4', 21),
 ('/home/youse/viva-data/5.mp4', 24),
 ('/home/youse/viva-data/5.mp4', 28),
 ('/home/youse/viva-data/5.mp4', 34),
 ('/home/youse/viva-data/5.mp4', 36),
 ('/home/youse/viva-data/5.mp4', 42),
 ('/home/youse/viva-data/5.mp4', 43),
 ('/home/youse/viva-data/5.mp4', 45),
 ('/home/youse/viva-data/5.mp4', 46),
 ('/home/youse/viva-data/5.mp4', 48),
 ('/home/youse/viva-data/5.mp4', 49),
 ('/home/youse/viva-data/5.mp4', 53),
 ('/home/youse/viva-data/5.mp4', 55),
 ('/home/youse/viva-data/5.mp4', 57),
 ('/home/youse/viva-data/5.mp4', 59),
 ('/home/youse/viva-data/5.mp4', 61),
 ('/home/youse/viva-data/5.mp4', 65),
 ('/home/youse/viva-data/5.mp4', 66),
 ('/home/youse/viva-data/5.mp4', 67),
 ('/home/youse/viva-data/5.mp4', 68),
 ('/home/youse